In [4]:
import pandas as pd
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
import IPython.display
import random
import warnings
import os
from PIL import Image
import pathlib
import csv
# sklearn Preprocessing
from sklearn.model_selection import train_test_split
#Keras
import keras
import warnings
warnings.filterwarnings('ignore')
from keras import layers
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras.optimizers import SGD


Collab için drive işlemleri

In [6]:
#drive tanımlaması yapılmak zorunda
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [7]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

 Her ses işlenip spectrogramları png olarak kaydediyor.

In [36]:

classes = 2
#current_path = r"C://Users//Beyza//Desktop//16000_pcm_speeches//Train"
current_path = "drive/My Drive/Colab Notebooks/16000_pcm_speeches/Train"
for i in range (classes):
    pathlib.Path(f'img_data/{i}').mkdir(parents=True, exist_ok=True) #for images
    path = os.path.join(current_path + '/' + str(i)) 
    print("data path:",path)
    datapath = os.listdir(path)  
   # print(i,".dosya path->",datapath)
    for path_new in datapath:
        path_name =  path +"/" +path_new   
        y, sr = librosa.load(path_name, mono=True, duration=5)
        #print(y.shape)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig((f'img_data/{i}/{path_new[:-3].replace(".", "")}.png'))    #Kaydet
        plt.clf()

data path: drive/My Drive/Colab Notebooks/16000_pcm_speeches/Train/0
data path: drive/My Drive/Colab Notebooks/16000_pcm_speeches/Train/1


<Figure size 432x288 with 0 Axes>

In [37]:
#For  dataset
import splitfolders 

splitfolders.ratio('./img_data/', output="./data", seed=1337, ratio=(.8, .2)) # default values
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator( #(X_train , y_train return)
        rescale=1./255,       # all our pixel values are in range (0,1)
        shear_range=0.2,      #to apply some random tranfromations
        zoom_range=0.2,       
        horizontal_flip=True) 

test_datagen = ImageDataGenerator( #(X_test , y_test return)
        rescale=1./255,       # all our pixel values are in range (0,1)
        shear_range=0.2,      #to apply some random tranfromations
        zoom_range=0.2,      
        horizontal_flip=True) 

Copying files: 3000 files [00:00, 6816.85 files/s]


In [38]:
#data generators

#class-mode -> yapılacak sınıflandırmaya göre uygun olanı seçilmelidir. (binary - categorical)     
training_set = train_datagen.flow_from_directory( 
    './data/train',
    target_size=(64, 64),
    batch_size=1,
    class_mode='binary',
    shuffle = False)

test_set = test_datagen.flow_from_directory(
    './data/val',
    target_size=(64, 64),
    batch_size=1,
    class_mode='binary',
    shuffle = False )



Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [39]:
model = Sequential()
input_shape=(64, 64, 3)                                              #1st hidden layer
model.add(Conv2D(32, (3, 3), strides=(2, 2), input_shape=input_shape))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))                                         #2nd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))                                         #3rd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))                                         #Flatten
model.add(Flatten())
model.add(Dropout(rate=0.5))                                          #Add fully connected layer.
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))                                          #Output layer
model.add(Dense(10))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 31, 31, 32)        896       
_________________________________________________________________
average_pooling2d_3 (Average (None, 15, 15, 32)        0         
_________________________________________________________________
activation_5 (Activation)    (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
average_pooling2d_4 (Average (None, 7, 7, 64)          0         
_________________________________________________________________
activation_6 (Activation)    (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 64)         

In [40]:
epochs = 200
batch_size = 8
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(optimizer="sgd",  loss ="sparse_categorical_crossentropy", metrics=['accuracy'])

In [41]:
model.fit_generator(
        training_set,
        steps_per_epoch= len(training_set), 
        epochs=50,
        validation_data=test_set,
        validation_steps= len(test_set), 
         )

Epoch 1/50
2400/2400 [==============================] - 18s 7ms/step - loss: 0.9239 - accuracy: 0.4658 - val_loss: 0.7083 - val_accuracy: 0.5000
Epoch 2/50
2400/2400 [==============================] - 17s 7ms/step - loss: 0.7506 - accuracy: 0.4698 - val_loss: 0.7160 - val_accuracy: 0.5000
Epoch 3/50
2400/2400 [==============================] - 17s 7ms/step - loss: 0.7308 - accuracy: 0.4904 - val_loss: 0.7382 - val_accuracy: 0.5000
Epoch 4/50
2400/2400 [==============================] - 17s 7ms/step - loss: 0.7243 - accuracy: 0.5026 - val_loss: 0.6661 - val_accuracy: 0.5000
Epoch 5/50
2400/2400 [==============================] - 17s 7ms/step - loss: 0.6605 - accuracy: 0.6202 - val_loss: 0.4727 - val_accuracy: 0.7850
Epoch 6/50
2400/2400 [==============================] - 17s 7ms/step - loss: 0.5849 - accuracy: 0.7089 - val_loss: 0.6098 - val_accuracy: 0.6417
Epoch 7/50
2400/2400 [==============================] - 17s 7ms/step - loss: 0.5006 - accuracy: 0.7704 - val_loss: 0.4895 - val_ac

In [42]:
#Model Evaluation
model.evaluate_generator(generator=test_set, steps=200)

[0.05806991457939148, 0.9850000143051147]

In [43]:
#test veri setini değiştiriyoruz. Ama önce sıfırlanması gerekiyor
test_set.reset()
pred = model.predict_generator(test_set, verbose=1) #steps= len(test_set)

600/600 [==============================] - 3s 5ms/step


In [45]:
predicted_class_indices=np.argmax(pred,axis=1)
#print(predicted_class_indices)

labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predictions = predictions[:600]
filenames=test_set.filenames
#print(filenames)

In [46]:
print(len(filenames), len(predictions))


600 600


Karşılaştırmalar için tablo oluşturuldu.

In [47]:
import csv
results=pd.DataFrame({"Filename" :filenames,
                      "Predictions":predictions})
                     #orient='index')
    
results.to_csv("prediction_results.csv",index=False)